# 패키지 불러오기 및 함수 정의

In [94]:
import os
import pandas as pd

API_KEY = 

import requests
import re

import time
import json


from tqdm import tqdm
tqdm.pandas()
from dotenv import load_dotenv
load_dotenv()

False

In [95]:
#카테고리별 요약
def mid_clsf_cat(text, retries = 3, attempt = 1):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    messages = [
        {"role": "system",
         "content": f"""
            너는 패션 회사 분류 AI야. 리뷰를 보고 알맞은 카테고리로 분류하는 일을 하고 있어. 다음 리뷰를 보고 100% 부합하는 카테고리를 모두 찾아줘. 주어진 카테고리 내에서만 반환해.

            #카테고리 목록
            - 가격
            - 디자인
            - 품질
            - TPO
            - 배송/재고
            - 광고

            #카테고리 설명 :
            - 가격: 제품이나 서비스의 가격과 그에 따른 가치 전반을 다루는 카테고리로, 합리적인 가격이나 가성비에 대한 평가부터 할인 및 프로모션을 통한 구매 경험까지 포함합니다.
            - 디자인: 제품의 시각적 요소와 착용감을 전반적으로 다루는 카테고리로, 사이즈와 핏에 따른 착용감, 독특한 디자인과 패턴, 매력적인 컬러까지 포함하여 제품의 외관과 착용 만족도를 모두 아우릅니다.
            - 품질: 제품의 소재, 마감, 기능성, 수선 및 관리까지 포함하여 전반적인 품질과 내구성을 다루는 카테고리입니다. 이는 사용된 재료의 특성, 세부적인 마감 처리, 실용적인 기능, 그리고 제품을 오래 사용하기 위한 관리 방법 등을 포괄합니다.
            - TPO: 특정한 시간, 장소, 상황에 맞는 적절한 복장과 스타일링을 다루는 카테고리로, 개인의 취향이나 최신 유행을 고려하여 다양한 룩을 연출하는 것을 포함합니다. 이는 옷을 입는 시간대, 장소, 행사 성격에 따라 스타일을 선택하거나, 다른 아이템과의 매칭으로 원하는 분위기를 내는 것을 의미합니다.
            - 배송/재고: 제품의 구매 및 재고 상황부터 배송 속도와 포장 상태, 그리고 반품 및 교환 경험까지 포함하여, 구매 과정 전반의 만족도를 다루는 카테고리입니다. 이는 재고 유무에 따른 구매 경험, 신속하고 안전한 배송과 포장, 그리고 반품이나 교환 과정에서의 고객 경험을 포괄합니다.
            - 광고: 광고 카테고리는 제품과 관련된 광고 요소를 다룹니다. 여기에는 광고 이미지와 실물의 차이 또는 유사성, 유명인이나 캐릭터의 등장, 프로모션 경험 등이 포함됩니다.

            #단계
            1. 리뷰 분석: 리뷰 내용을 읽고, 각 카테고리에 부합하는지 판단.
            2. 정확한 카테고리 매칭: 카테고리 설명에 따라, 주어진 설명과 정확히 일치하는 경우에만 해당 카테고리로 분류.
            3. 답변 형식으로 반환: 각 카테고리를 쉼표로 구분해 반환.

            # 답변 형식
            카테고리1, 카테고리2, ...

            #주의
            착화감/착용감은 디자인 카테고리에만 해당됨. 품질로 분류하지 않음.
            TPO 카테고리는 ‘제품을 사용하는 구체적인 시간, 장소, 상황’이 명시된 경우에만 해당됨. 단순히 '시간', '장소', '상황' 단어가 포함되었다고 TPO로 분류하지 말 것.
            사진,실물 키워드가 나오면 광고에 속함
            재구매 키워드는 배송/재고에 속함
            사람을 지칭하는 대명사는 광고에 속함
            카테고리와 일치하지 않는 내용은 반환하지 말 것. 각 리뷰의 내용이 설명과 완전히 일치하는 경우에만 해당 카테고리로 분류.

         """},
         {"role": "user", "content": f"리뷰 내용 : {text}"}
         ]

    data = {
        "model": "gpt-4o-mini",
        "messages": messages,
        "temperature":0.2,
        "max_tokens" : 2056,
        "top_p":1,
        "frequency_penalty":1.0,
        "presence_penalty":1.0
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # HTTP 에러 발생 시 예외 발생

        # 응답 내용 확인
        response_text = response.json().get('choices', [{}])[0].get('message', {}).get('content', '')

        # 응답이 비어 있거나 공백만 있는 경우
        if not response_text.strip():
            print(f"빈 응답 확인됨. 재시도 중... (시도 {attempt}/{retries})")
            if attempt < retries:
                time.sleep(2)  # 2초 대기 후 재시도
                return mid_clsf_cat(text, retries, attempt + 1)
            else:
                return "재시도 횟수 초과로 분류 실패"

        # 유효한 응답 반환
        return response_text

    except requests.exceptions.RequestException as e:
        print(f"요청 실패: {e}")
        if attempt < retries:
            time.sleep(2)  # 2초 대기 후 재시도
            return mid_clsf_cat(text, retries, attempt + 1)
        else:
            return f"재시도 횟수 초과로 요청 실패: {e}"

    except json.JSONDecodeError:
        print("JSONDecodeError: 응답이 JSON 형식이 아님")
        if attempt < retries:
            time.sleep(2)  # 2초 대기 후 재시도
            return mid_clsf_cat(text, retries, attempt + 1)
        else:
            return "재시도 횟수 초과로 JSON 디코딩 실패"

In [96]:
#파일 불러오기

mid = pd.read_excel('D:/업무 파일/한중팀/1.CRD/RAD 버전업/2. 상세_시즌/정답지(1).xlsx',sheet_name='정답지-1733198495270')
mid

,style_code,product_name,brand_name,channel,review_date,review_text
0,CVCKE4901N,[촉감니트] 울혼방 베이직 가디건,클라비스,shoplink,2024-11-01,착용 시 왼쪽 가슴에 부착되어 있는 클라비스 뱃지 떨어짐.
1,CVCKE4901N,[촉감니트] 울혼방 베이직 가디건_CVCKE4901N,클라비스,elandmall,2024-11-02,색깔 보는것처럼 쨍해요!! 예뻐요. 그런데 저한테는 약간 큰편이예요
2,CVCKE4901N,[촉감니트] 울혼방 베이직 가디건,클라비스,shoplink,2024-11-03,첫번쨰 단추 불량
3,CVCKE4901N,[촉감니트] 울혼방 베이직 가디건,클라비스,shoplink,2024-11-04,와팬 떨어짐
4,CVCKE4901N,[촉감니트] 울혼방 베이직 가디건,클라비스,shoplink,2024-11-04,앞판 왼쪽 밑 오염
...,...,...,...,...,...,...
1767,SPJPE4VG01,퍼 탈착 푸퍼,스파오,shoplink,2024-11-29,1. 뒷면 기준 오른쪽 팔 부분 점 얼룩 2. 정면 기준 오른쪽 주머니 윗 부분 색 오염
1768,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,스파오,mall,2024-11-30,매장에서 입어보고 구매했어요! 사이즈는 M이 넉넉하게 잘 맞았지만 S가 부해보이지 ...
1769,SPJPE4VG01,퍼 탈착 푸퍼,스파오,shoplink,2024-11-30,"탈착칼라 왼쪽 첫번째 단추 탈락, 오른쪽도 탈락 직전입니다. 수선 부탁드립니다."
1770,SPJPE4VG01,퍼 탈착 푸퍼,스파오,shoplink,2024-11-30,상품 하단 조임끈이 박음질이 되어있어 판매가 어렵습니다. 빠른 수선 부탁드립니다. ...


# 중분류 분류 확인

In [97]:
from tqdm import tqdm
import time
import numpy as np

cat_6p = mid[['style_code','product_name','channel','review_date','review_text']]
cat_6p['6P'] = cat_6p.progress_apply(
    lambda row: '품질' if row['channel'] == 'shoplink' else mid_clsf_cat(row['review_text']),
    axis=1
)

100%|██████████| 1772/1772 [14:14<00:00,  2.07it/s]


In [98]:
# 6P 고정
import numpy as np

categories = ['가격', '디자인', '품질', 'TPO', '배송/재고', '광고']

for category in categories:
    cat_6p[category] = False

In [99]:
for index, row in cat_6p.iterrows():
    categories = row['6P'].split(', ')
    for category in categories:
        if category in cat_6p.columns:
            cat_6p.at[index, category] = True

In [100]:
# cat_6p.fillna(False, inplace=True)
df_mid = pd.melt(
    cat_6p,
    id_vars=['style_code', 'product_name', 'channel', 'review_text'],  # 유지할 열
    value_vars=['가격', '디자인', '품질', 'TPO', '배송/재고', '광고'],  # melt할 열
    var_name='6P',  # 기존에 'variable'이었던 컬럼 이름을 '6P'로 변경
    value_name='6P_tmp'  # 기존에 'value'였던 컬럼 이름을 '6P_tmp'로 변경
)
df_mid.sort_values(by=['style_code','review_text'], inplace=True)
df_mid

,style_code,product_name,channel,review_text,6P,6P_tmp
14,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,가격,True
1786,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,디자인,False
3558,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,품질,False
5330,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,TPO,False
7102,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,배송/재고,False
...,...,...,...,...,...,...
3535,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,디자인,True
5307,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,품질,False
7079,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,TPO,False
8851,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,배송/재고,False


# 소분류 분류 확인

### 소분류 프롬프트

In [101]:
def gpt_classification(text, category, system_message,detail_cat,retries = 5, attempt =1):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    messages = [
        {"role": "system", "content": system_message}, 
        {"role": "user", "content": f"{text}"}
    ]
    
    data = { 
        "model": "gpt-4o-mini", 
        "messages": messages, 
        "temperature": 0.3, 
        "max_tokens": 2056, 
        "top_p": 1, 
        "frequency_penalty": 1.0, 
        "presence_penalty": 1.0,
        "response_format" : {"type":"json_object"}
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # 요청 에러 발생 시 예외 발생

        result_content = response.json().get('choices', [{}])[0].get('message', {}).get('content', '{}')
        result_content = re.sub(r"```json|```", "", result_content).strip()        
        parsed_result = json.loads(result_content)  # JSON 파싱

        # 'subcategory'와 'posi' 값을 처리
        points_posi_pairs = []
        for point_obj in parsed_result.get('contents', {}).get('points', []):
            subcategory = point_obj.get('subcategory', 'No subcategory')
            posi = point_obj.get('posi', 'No posi')

            if str(subcategory) not in detail_cat:
                print(f"subcategory {subcategory} is not in detail category, Retrying....")
                if attempt < retries:
                    return gpt_classification(text, category, system_message, detail_cat, retries, attempt + 1)
                else:
                    return "재시도 횟수 초과"

            if str(posi) not in ['긍정','부정']:
                print('긍정, 부정 분류 오류')
                if attempt < retries:
                    return gpt_classification(text, category, system_message, detail_cat, retries, attempt + 1)
                else:
                    return "재시도 횟수 초과"            

            points_posi_pairs.append({'subcategory': subcategory, 'posi': posi})

        unique_points_posi_pairs = []
        seen_subcategories = set()

        for pair in points_posi_pairs:
            if pair['subcategory'] not in seen_subcategories:
                unique_points_posi_pairs.append(pair)
                seen_subcategories.add(pair['subcategory'])

        return unique_points_posi_pairs  # 카테고리와 긍부정 쌍을 리스트로 반환

    except json.JSONDecodeError:
        print("JSONDecodeError: 응답 본문을 JSON으로 파싱할 수 없습니다.")
        print("Response text:", response.text)
        return "JSON 파싱에 실패했습니다. 응답 본문이 예상된 형식이 아닙니다."

    except requests.exceptions.RequestException as e:
        return f"API 요청에 실패했습니다. 에러 메시지: {str(e)}"



In [102]:

# 중분류 = '가격'
def price_clsf(text, category):
    price_lst = ['가격','할인']
    system_message = """
    리뷰 텍스트를 입력 받으면, 해당 리뷰에 대한 카테고리와 카테고리에 대한 긍정,부정을 분류하세요. 카테고리는 카테고리 목록에서 선택하세요.

    # Steps

    1. 텍스트 분석: 입력된 리뷰 텍스트를 분석하여 주요 주제와 관련 정보를 식별합니다.
    2. 카테고리 분류: 분석한 내용을 바탕으로 리뷰의 주된 카테고리를  상세 카테고리 설명을 참고해 상세 카테고리 목록에서 선택합니다. 
    3. 긍정,부정 분류: 각 카테고리에 대해 긍정적인 어조인지, 부정적인 어조인지 확인해 긍정 or 부정 둘 중 하나로 분류합니다. 비꼬는 어조는 부정적인 어조입니다.

    #상세 카테고리 목록
    가격
    할인

    #상세 카테고리 설명
    가격 : 가격도 저렴하니 가성비 넘쳐요~와 같은 가격과 관련된 카테고리
    할인 : 이번에 뿌린 할인 쿠폰 적용해서 저렴하게 구매했어요~와 같이 물건을 싸게 판매하여 사람들이 구매하도록 유도하는 프로모션 '할인'과 관련된 카테고리. '가성비' 키워드는 여기에 속하지 않음.

    # Output Format

    결과는 JSON 형식으로 반환됩니다. 각 카테고리와 해당 긍부정 점수를 포함해야 합니다.
    ```json
    {
            "contents": {
                "points": [
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                ...
                ]
            }
            }

    #제약 조건
    긍정, 부정만 반환해. 중립은 반환하지마. 숫자가 아닌 긍정 or 부정 값만 반환해
    일반적인 저렴한 가격은 '할인'에 포함되지 않아

    """

    return gpt_classification(text, category, system_message,price_lst)

# 중분류 = '디자인'
def pattern_clsf(text, category):
    pattern_lst = ['사이즈', '디자인','컬러','핏/착용감']
    system_message = """
    리뷰 텍스트를 입력 받으면, 해당 리뷰에 대한 카테고리와 카테고리에 대한 긍정,부정을 분류하세요. 카테고리는 카테고리 목록에서 선택하세요.

    # Steps

    1. 텍스트 분석: 입력된 리뷰 텍스트를 분석하여 주요 주제와 관련 정보를 식별합니다.
    2. 카테고리 분류: 분석한 내용을 바탕으로 리뷰의 주된 카테고리를  상세 카테고리 설명을 참고해 상세 카테고리 목록에서 선택합니다. 
    3. 긍정/부정 분류: 각 카테고리에 대해 긍정적인 어조인지, 부정적인 어조인지 확인해 긍정 or 부정 둘 중 하나로 분류합니다. 비꼬는 어조는 부정적인 어조입니다.

    #상세 카테고리 목록
    사이즈
    디자인
    컬러
    핏/착용감

    #상세 카테고리 설명
    사이즈 : 제품의 크기나 치수를 의미해. 의류나 신발의 경우, 사이즈는 착용자의 신체에 얼마나 잘 맞는지를 결정하는 중요한 요소야.
    디자인 : 이번에 프린팅/ 체크패턴 너무 이쁘네요~ 와 같은 제품에 적용된 시각적인 요소로, 디자인의 개성과 특징을 나타내는 중요한 부분이야.제품 자체의 디자인부터 티셔츠나 후드티 등에 프린트된 이미지, 로고, 텍스트,원단 패턴, 부자재 디테일등의 상세한 디자인 요소를 의미해
    컬러 : 이번에 00색 너무 이뻐요. 색감이 오묘한게 너무 예쁘네요~와 같은 컬러와 관련된 카테고리 
    핏/착용감 : 오버핏이라 여리여리해보여요, 안 신은 것 같은 착용감 최고~와 같은 제품이 착용자의 몸에 어떻게 맞는지를 나타내는 카테고리야. 루즈핏,슬림핏,오버핏과 같은 fit관련 리뷰, 상품을 착용했을때의 착화/착용감과 관련된 카테고리야. 사이즈와는 달라

    # Output Format

    결과는 JSON 형식으로 반환됩니다. 각 카테고리와 해당 긍부정 점수를 포함해야 합니다.
    ```json
    {
            "contents": {
                "points": [
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                ...
                ]
            }
            }

    #제약 조건
    긍정, 부정만 반환해. 중립은 반환하지마. 숫자가 아닌 긍정 or 부정 값만 반환해
    """
    return gpt_classification(text, category, system_message,pattern_lst)

# 중분류 = '품질'
def product_clsf(text, category):
    product_lst = ['소재','마감','기능성','수선/관리']
    system_message = """
    리뷰 텍스트를 입력 받으면, 해당 리뷰에 대한 카테고리와 카테고리에 대한 긍정,부정을 분류하세요. 카테고리는 카테고리 목록에서 선택하세요.

    # Steps

    1. 텍스트 분석: 입력된 리뷰 텍스트를 분석하여 주요 주제와 관련 정보를 식별합니다.
    2. 카테고리 분류: 분석한 내용을 바탕으로 리뷰의 주된 카테고리를  상세 카테고리 설명을 참고해 상세 카테고리 목록에서 선택합니다. 
    3. 긍정/부정 분류: 각 카테고리에 대해 긍정적인 어조인지, 부정적인 어조인지 확인해 긍정 or 부정 둘 중 하나로 분류합니다. 비꼬는 어조는 부정적인 어조입니다.

    #상세 카테고리 목록
    소재
    마감
    기능성
    수선/관리

    #상세 카테고리 설명
    소재 : 린넨 소재라 여름에 시원해요~이런 제품에 사용된 원재료나 재질을 의미해. 재질, 원단 모두 소재에 속해
    마감 : 제품의 세부적인 마감 처리를 의미해. 이는 제품의 완성도와 퀄리티를 결정하지. 예를 들어, 옷의 스티칭 퀄리티, 단추나 지퍼의 품질, 가장자리 마감, 자수나 장식 등의 세부 요소들이 디테일에 해당해. 마감이 우수하면 제품의 전체적인 품질과 가치를 높여줘.
    기능성 : 제품이 제공하는 실용적인 기능이나 성능을 의미해.예를 들어, 방수 재킷의 방수 능력, 운동화의 쿠셔닝과 통기성, 착용자의 체온을 자동으로 조절하는 스마트 섬유 기능, 냄세 억제하는 방취 기능 등 기능성 의류나 첨단 기술이 적용된 것을 의미해.
    수선/관리 : 00재질이라 관리가 좀 힘들어요;; or 기장이 좀 길어서 수선해 입었습니다 이런 제품을 오래 사용하기 위해 필요한 관리 방법이나 수선 가능성을 의미해.

    # Output Format

    결과는 JSON 형식으로 반환됩니다. 각 카테고리와 해당 긍부정 점수를 포함해야 합니다.
    ```json
    {
            "contents": {
                "points": [
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                ...
                ]
            }
            }

    #제약 조건
    긍정, 부정만 반환해. 중립은 반환하지마. 숫자가 아닌 긍정 or 부정 값만 반환해
    """
    return gpt_classification(text, category, system_message,product_lst)

# 중분류 = 'TPO'
def place_clsf(text, category):
    place_lst = ['TPO','스타일링']
    system_message = """
    리뷰 텍스트를 입력 받으면, 해당 리뷰에 대한 카테고리와 카테고리에 대한 긍정,부정을 분류하세요. 카테고리는 카테고리 목록에서 선택하세요.

    # Steps

    1. 텍스트 분석: 입력된 리뷰 텍스트를 분석하여 주요 주제와 관련 정보를 식별합니다.
    2. 카테고리 분류: 분석한 내용을 바탕으로 리뷰의 주된 카테고리를  상세 카테고리 설명을 참고해 상세 카테고리 목록에서 선택합니다. 
    3. 긍정/부정 분류: 각 카테고리에 대해 긍정적인 어조인지, 부정적인 어조인지 확인해 긍정 or 부정 둘 중 하나로 분류합니다. 비꼬는 어조는 부정적인 어조입니다.


    #상세 카테고리 목록
    TPO
    스타일링


    #상세 카테고리 설명
    TPO :  '시간(Time), 장소(Place), 상황(Occasion)'의 약자로, 특정한 상황에 맞게 적절한 복장을 선택하는 것을 의미해. 이는 옷을 입을 때 단순히 스타일이나 트렌드만을 고려하는 것이 아니라, 그 옷을 입는 시간대, 장소, 그리고 그 장소에서의 역할이나 행사 성격까지 고려하여 스타일링하는 것을 말해.
    스타일링 : 가을 분위기 내기 좋네요, 00 상품이랑 매칭하니까 이쁘네요, 00룩과 어울려요 같은 개인의 취향이나 최신 유행에 따라 옷을 선택하는 것을 말해. 스타일, 스타일링이 있다면 대부분 이 카테고리에 속해

    # Output Format

    결과는 JSON 형식으로 반환됩니다. 각 카테고리와 해당 긍부정 점수를 포함해야 합니다.
    ```json
    {
            "contents": {
                "points": [
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                ...
                ]
            }
            }

    #제약 조건
    제품을 사용하는 시간/장소/상황이 나타나있지 않다면 TPO로 분류해선 안됩니다. 
    긍정, 부정만 반환해. 중립은 반환하지마. 숫자가 아닌 긍정 or 부정 값만 반환해
    """
    return gpt_classification(text, category, system_message,place_lst)

# 중분류 = '배송/재고'
def process_clsf(text, category):
    process_lst = ['구매/재고','배송/포장','반품/포장']
    system_message = """
    리뷰 텍스트를 입력 받으면, 해당 리뷰에 대한 카테고리와 카테고리에 대한 긍정,부정을 분류하세요. 카테고리는 카테고리 목록에서 선택하세요.

    # Steps

    1. 텍스트 분석: 입력된 리뷰 텍스트를 분석하여 주요 주제와 관련 정보를 식별합니다.
    2. 카테고리 분류: 분석한 내용을 바탕으로 리뷰의 주된 카테고리를  상세 카테고리 설명을 참고해 상세 카테고리 목록에서 선택합니다. 
    3. 긍정/부정 분류: 각 카테고리에 대해 긍정적인 어조인지, 부정적인 어조인지 확인해 긍정 or 부정 둘 중 하나로 분류합니다. 비꼬는 어조는 부정적인 어조입니다.

    #상세 카테고리 목록
    구매/재고
    배송/포장
    반품/교환


    #상세 카테고리 설명
    구매/재고 : "재고가 없어서 기다리다가 이번에 구매했어요.", "저번에 하나 사고 좋아서 재구매했습니다." 같은, 재고 상황과 구매 및 재구매 경험을 나타내는 카테고리
    배송/포장 : "배송이 정말 빠르네요!", "포장이 정말 최고예요.", "포장이 다 구겨져서 왔어요."와 같은 배송 속도와 포장 상태에 대한 경험을 나타내는 카테고리.
    반품/교환 : "저번에 오염되어서 반품했는데, 이번엔 실밥이 터졌네요."와 같은 반품 및 교환 과정에서의 경험을 나타내는 카테고리.

    # Output Format

    결과는 JSON 형식으로 반환됩니다. 각 카테고리와 해당 긍부정 점수를 포함해야 합니다.
    ```json
    {
            "contents": {
                "points": [
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                ...
                ]
            }
            }
    #제약 조건
    긍정, 부정만 반환해. 중립은 반환하지마. 숫자가 아닌 긍정 or 부정 값만 반환해
    """
    return gpt_classification(text, category, system_message,process_lst)
    
# 중분류 = '광고'
def promotion_clsf(text, category):
    promotion_lst = ['캐릭터/인물','이벤트','사진/실물']
    system_message = """
    리뷰 텍스트를 입력 받으면, 해당 리뷰에 대한 카테고리와 카테고리에 대한 긍정,부정을 분류하세요. 카테고리는 카테고리 목록에서 선택하세요.

    # Steps

    1. 텍스트 분석: 입력된 리뷰 텍스트를 분석하여 주요 주제와 관련 정보를 식별합니다.
    2. 카테고리 분류: 분석한 내용을 바탕으로 리뷰의 주된 카테고리를  상세 카테고리 설명을 참고해 상세 카테고리 목록에서 선택합니다. 
    3. 긍정/부정 분류: 각 카테고리에 대해 긍정적인 어조인지, 부정적인 어조인지 확인해 긍정 or 부정 둘 중 하나로 분류합니다. 비꼬는 어조는 부정적인 어조입니다.

    #상세 카테고리 목록
    캐릭터/인물
    이벤트
    사진/실물


    #상세 카테고리 설명
    캐릭터/인물 : 아이유가 입은거 이뻐서 샀어요, 이번에 캐릭터랑 콜라보한거 귀여워서 샀어요 이렇게 유명인,인물, 캐릭터와 관련된 카테고리야. 광고에 등장하는 인물/캐릭터 일 수 있고, 해당 제품을 착용한 유명인이 언급되면 해당 카테고리에 속해. 또한 착용하는 '인물'에 대한 정보가 나와있다면 해당 카테고리에 속해
    이벤트 : 이번에 이랜드위크/무신사 이벤트/ 에이블리 프로모션/ 쿠폰 이벤트 .... 가 있어서 저렴하게 샀어요 같은 소비자 참여를 유도하는 특별한 활동이나 행사를 말해.신제품 출시 행사, 팬미팅, 온라인 챌린지, 체험 부스 등이 이에 속해. 여기서 중요한 점은 이벤트 자체가 광고의 한 형태라는 것이야. 브랜드나 제품에 대한 관심과 참여를 높이기 위해 기획된 프로모션 활동이야.
    사진/실물 : 실물이 훨씬 이뻐요, 사진이랑 색이 다르네요 이런 실물과 사진 관련 카테고리야

    # Output Format

    결과는 JSON 형식으로 반환됩니다. 각 카테고리와 해당 긍부정 점수를 포함해야 합니다.
    ```json
    {
            "contents": {
                "points": [
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                {
                    "subcategory": "상세 카테고리",
                    "posi":"긍정 or 부정"
                },
                ...
                ]
            }
            }

    #제약 조건
    긍정, 부정만 반환해. 중립은 반환하지마. 숫자가 아닌 긍정 or 부정 값만 반환해
    """
    return gpt_classification(text, category, system_message,promotion_lst)


category_functions = {
    '가격': price_clsf,
    '디자인': pattern_clsf,
    '품질': product_clsf,
    'TPO': place_clsf,
    '배송/재고': process_clsf,  
    '광고': promotion_clsf
}

# 함수 실행 함수
def clsf_rev(row):
    rev_txt = row['Review Text']
    mid_cat = row['6P'] 
    clsf = row['6P_tmp']
    
    # 카테고리에 따른 분류 함수 호출
    if clsf == True and mid_cat == '가격':
        return price_clsf(rev_txt, mid_cat)
    elif clsf == True and mid_cat == '디자인':
        return pattern_clsf(rev_txt, mid_cat)
    elif clsf == True and mid_cat == '품질':
        return product_clsf(rev_txt, mid_cat)
    elif clsf == True and mid_cat == 'TPO':
        return place_clsf(rev_txt, mid_cat)
    elif clsf == True and mid_cat == '배송/재고':
        return process_clsf(rev_txt, mid_cat)
    elif clsf == True and mid_cat == '광고':
        return promotion_clsf(rev_txt, mid_cat)
    else:
        return " "
    

### 소분류 실행 코드

In [103]:
df_mid.rename(columns={'review_text':'Review Text'},inplace=True)
test_mid = df_mid.copy(deep=True)
test_mid.reset_index(drop=True,inplace=True)
test_mid['Review Text'] = test_mid['Review Text'].fillna('').astype(str)

#CS가 아니면 분류 시작
openmarket_df = test_mid[test_mid['channel']!='shoplink']
shoplink_df = test_mid[test_mid['channel']=='shoplink']

openmarket_df['api_result'] = openmarket_df.progress_apply(lambda x : clsf_rev(x),axis=1)

In [106]:
result = []
for idx, row in openmarket_df.iterrows():
    style_code = row.get('style_code')
    product_name = row.get('product_name')
    channel = row.get('channel')
    review_text = row.get('Review Text')
    six_p = row.get('6P')
    six_tmp = row.get('6P_tmp')
    if six_tmp:
        api_result = row.get('api_result')
        for api in api_result:
            try:
                sub_category = api.get('subcategory')
                positive = api.get('posi')
                result.append({
                    'style_code' : style_code,
                    'product_name' : product_name,
                    'channel' : channel,
                    'review_text' : review_text,
                    'six_p' : six_p,
                    'six_tmp' : six_tmp,
                    'sub_category' : sub_category,
                    'positive' : positive
                })
            except:
                pass
    else:
        result.append({
                'style_code' : style_code,
                'product_name' : product_name,
                'channel' : channel,
                'review_text' : review_text,
                'six_p' : six_p,
                'six_tmp' : six_tmp,
                'sub_category' : '없음',
                'positive' : '없음'
        })

cs_result = []
for idx, row in shoplink_df.iterrows():
    style_code = row.get('style_code')
    product_name = row.get('product_name')
    channel = row.get('channel')
    review_text = row.get('Review Text')
    six_p = row.get('6P')
    six_tmp = row.get('6P_tmp')
    if six_tmp:
        api_result = row.get('api_result')
        for api in api_result:
            try:
                sub_category = '수선/관리'
                positive = '부정'
                cs_result.append({
                    'style_code' : style_code,
                    'product_name' : product_name,
                    'channel' : channel,
                    'review_text' : review_text,
                    'six_p' : six_p,
                    'six_tmp' : six_tmp,
                    'sub_category' : sub_category,
                    'positive' : positive
                })
            except:
                pass
    else:
        cs_result.append({
                'style_code' : style_code,
                'product_name' : product_name,
                'channel' : channel,
                'review_text' : review_text,
                'six_p' : six_p,
                'six_tmp' : six_tmp,
                'sub_category' : '없음',
                'positive' : '없음'
        })


In [107]:
review_df = pd.DataFrame(result)
cs_df = pd.DataFrame(cs_result)
cs_df.drop_duplicates(subset=['style_code','review_text','six_p'],inplace=True)
cs_df.reset_index(drop=True,inplace=True)

In [108]:
#concat 파트
final_df = pd.concat([review_df,cs_df],axis=0)
final_df.sort_values(by=['style_code','review_text','six_p'],inplace=True)
final_df.drop_duplicates(subset=['style_code', 'review_text', 'six_p','sub_category'], keep = 'first' ,inplace=True)
final_df.reset_index(drop=True, inplace=True)

# 한줄 요약 + 언급 횟수 + 비즈니스 액션

## 리뷰 집계 함수

In [109]:
def summary_groupby(df, separator='.'):
    tmp = df.copy(deep=True)
    group_columns = ['style_code','six_p', 'sub_category']
    join_column = 'review_text'
    
    # 그룹화 후 조건에 따라 review_text를 '없음' 또는 연결된 문자열로 설정
    grouped_df = (
        tmp.groupby(group_columns)[join_column]
        .apply(lambda x: '없음' if (x.name[-1] == '없음') else separator.join(x.dropna().astype(str)))
        .reset_index()
    )

    return grouped_df

col_ls = ['style_code','six_p','sub_category','review_text']
summ_df = final_df[col_ls]
summ_fre_act = summary_groupby(summ_df)
summ_fre_act = summ_fre_act[summ_fre_act['sub_category'] != '없음']

# 요약 프로세스 ver2

### 요약 프롬프트

In [110]:
import numpy as np

detail_cat_dic = {
    'price_summary' : '가격',
    'sale_summary' : '할인',
    'size_summary' : '사이즈',
    'design_summary' : '디자인',
    'color_summary' : '컬러',
    'fit_summary' : '핏/착용감',
    'fabric_summary' : '소재',
    'detail_summary' : '마감',
    'util_summary' : '기능성',
    'repair_summary' : '수선/관리',
    'tpo_summary' : 'TPO',
    'style_summary' : '스타일링',
    'stock_summary' : '구매/재고',
    'delivery_sumamry' : '배송/포장',
    'refund_summary' : '반품/교환',
    'person_summary' : '캐릭터/인물',
    'event_sumamry' : '이벤트',
    'picture_summary' : '사진/실물'
}

def summ_act(text,system_message,retries = 3, attempt = 1):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    messages = [
        {"role": "system", "content": system_message}, 
        {"role": "user", "content": text}
    ]
    
    data = { 
        "model": "gpt-4o-mini", 
        "messages": messages, 
        "temperature": 0.4, 
        "max_tokens": 2056, 
        "top_p": 1, 
        "frequency_penalty": 1.0, 
        "presence_penalty": 1.0,
        "response_format" : {"type":"json_object"}
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # 요청 에러 발생 시 예외 발생

        result_content = response.json().get('choices', [{}])[0].get('message', {}).get('content', '{}')
        result_content = re.sub(r"```json|```", "", result_content).strip()        
        parsed_result = json.loads(result_content)  # JSON 파싱

        # 'subcategory'와 'posi' 값을 처리
        points_posi_pairs = []
        for point_obj in parsed_result.get('contents', {}).get('summaries', []):
            freq = int(point_obj.get('freq', '0'))
            summary = point_obj.get('summary','No summary')

            if pd.isna(freq) or pd.isna(summary) or summary.strip() == "":
                print("반환 실패,,,재시도 중")
                if attempt < retries:
                    return summ_act(text,system_message,retries, attempt+1)
                else:
                    return "재시도 횟수 초과"
            points_posi_pairs.append({'freq' : freq,'summary': summary})

        if points_posi_pairs:
            max_pair = max(points_posi_pairs, key=lambda x: x['freq'])
            return [max_pair]  # 가장 freq가 높은 값을 리스트로 반환
        else:
            return []


    except json.JSONDecodeError:
        print("JSONDecodeError: 응답 본문을 JSON으로 파싱할 수 없습니다.")
        print("Response text:", response.text)
        return "JSON 파싱에 실패했습니다. 응답 본문이 예상된 형식이 아닙니다."

    except requests.exceptions.RequestException as e:
        return f"API 요청에 실패했습니다. 에러 메시지: {str(e)}"

def dynamic_summary(subc,review_text):
    matching_key = None
    for key,value in detail_cat_dic.items():
        if value == subc : 
            matching_key = key
            break

    if matching_key:
        if matching_key in globals():
            return globals()[matching_key](review_text)

        else:
            print(f"함수 '{matching_key}'가 정의되지 않았습니다.")
            return None




In [111]:
# 상세 카테고리 요약
#가격
def price_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '가격' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	가격 : 가격도 저렴하니 가성비 넘쳐요~와 같은 제품 가격과 관련된 내용

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """


    return summ_act(text, system_message)

#할인
def sale_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '할인' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	할인 : 물건을 싸게 판매하여 사람들이 구매하도록 유도하는 '할인' promotion과 관련된 카테고리. 

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#사이즈 
def size_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '사이즈' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	사이즈 : 제품의 크기,기장,치수를 의미해. 착용자의 신체에 얼마나 잘 맞는지를 나타내는 카테고리.

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#디자인 
def design_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '디자인' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	디자인 : 제품에 적용된 시각적인 요소로, 디자인의 개성과 특징을 나타내는 중요한 부분이야.제품 자체의 디자인부터 티셔츠나 후드티 등에 프린트된 이미지, 로고, 텍스트,원단 패턴, 부자재 디테일등의 상세한 디자인 요소를 의미해

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#컬러
def color_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '컬러' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	컬러 : 제품 색감과 관련된 카테고리. 

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#핏/착용감
def fit_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '핏/착용감' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	핏/착용감 : 제품이 착용자의 몸에 어떻게 맞는지를 나타내는 카테고리야. 00핏과 관련된 언급이 있다면 모두 여기에 속해. 착용감, 착화감 관련 언급도 모두 여기에 속해. size와 다른 fit/comfort 관점이야

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#소재
def fabric_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '소재' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘
    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	소재 : 제품에 사용된 원단과 소재,재질을 의미해. 

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#마감
def detail_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '디테일' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	마감 : 제품의 세부적인 마감 처리를 의미해. 이는 제품의 완성도와 퀄리티를 결정하지. 예를 들어, 옷의 스티칭 퀄리티, 단추나 지퍼의 품질, 가장자리 마감, 자수나 장식 등의 세부 요소들이 디테일에 해당해. 마감이 우수하면 제품의 전체적인 품질과 가치를 높여줘.

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#기능성
def util_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '기능성' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	기능성 : 제품이 제공하는 실용적인 기능이나 성능을 의미해.예를 들어, 방수 재킷의 방수 능력, 운동화의 쿠셔닝과 통기성, 착용자의 체온을 자동으로 조절하는 스마트 섬유 기능, 냄세 억제하는 방취 기능 등 기능성 의류나 첨단 기술이 적용된 것을 의미해.
    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#수선/관리
def repair_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '수선/관리' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	수선/관리 : 제품을 오래 사용하기 위해 필요한 관리 방법이나 수선과 관련된 카테고리. 고객이 스스로 수선하거나 관리하는 것 또한 여기에 속해

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#TPO
def tpo_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 'TPO' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	TPO :  '시간(Time), 장소(Place), 상황(Occasion)'의 약자로, 특정한 상황에 맞게 적절한 복장을 선택하는 것을 의미해. 옷을 입는 시간대, 장소, 그리고 그 장소에서의 역할이나 행사 성격까지 고려하여 착용하는 것을 말해.

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#스타일링
def style_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '스타일링' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	스타일링 : 개인의 취향이나 최신 유행에 따라 옷을 입는 것을 것을 말해. 어떤 분위기, 스타일로 코디했느냐를 의미해

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#구매/재고
def stock_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '구매/재고' 관점에서 요약하고,요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	구매/재고 : 재고 상황과 관련된 경험, 구매 및 재구매 경험을 나타내는 카테고리


    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#배송/포장
def delivery_sumamry(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '배송/포장' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	배송/포장 : 배송 관련 경험과 포장 상태에 대한 내용을 나타내는 카테고리

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#반품/교환
def refund_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '반품/교환' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	반품/교환 : 반품 및 교환 과정, 경험을 나타내는 카테고리.

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)


#캐릭터/인물
def person_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '캐릭터/인물' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	캐릭터/인물 광고에 등장하는 인물/캐릭터 일 수 있고, 해당 제품을 착용한 유명인이 언급되면 해당 카테고리에 속해. 또한 착용하는 '인물'에 대한 정보가 나와있다면 해당 카테고리에 속해 

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)


#이벤트
def event_sumamry(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '이벤트' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	이벤트 : 소비자 구매를 유도하는 특별한 활동이나 행사를 말해.신제품 출시 행사, 팬미팅, 온라인 챌린지, 체험 부스 등이 이에 속해. 여기서 중요한 점은 브랜드나 제품에 대한 관심과 참여를 높이기 위해 기획된 프로모션 활동이야.

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

#사진/실물
def picture_summary(text):
    system_message = """
    너는 패션 리뷰 요약 AI이자 비즈니스 컨설턴트야. 입력된 리뷰를 '사진/실물' 관점에서 요약하고, 요약 내용이 전체 리뷰에서 몇번 나타났는지 알려줘

    #데이터 입력 방식
    카테고리 : 카테고리
    리뷰 : 리뷰 내용
    
    #상세 카테고리 설명
	사진/실물 : 실물이 훨씬 이뻐요, 사진이랑 색이 다르네요 이런 실물과 사진 관련 카테고리야. 광고 행위의 한 종류야.

    #프로세스 
    1. 카테고리 확인: 주어진 카테고리와 직접적으로 연관된 내용만을 찾기 위해 리뷰를 분석.
    2. 관련 정보 추출: 리뷰에서 해당 카테고리에 완벽하게 부합하는 정보만 추출하고, 관련되지 않은 내용은 무시.
    3. 카테고리 관점 요약: 해당 카테고리와 직접적으로 연결된 정보만 바탕으로 리뷰를 한 줄로 요약.
    4. 언급 횟수 계산: 요약된 내용이 리뷰에서 몇 번 나타났는지 세기.
    5. 최다 언급 요약 선택: 언급 횟수가 가장 많은 요약을 선택.

    # Output Format

        결과는 JSON 형식으로 반환 됩니다. 요약 문장과, 언급 횟수를 반환합니다. 
        ```json
        {
                "contents": {
                    "summaries": [
                    {
                        "freq": "언급 횟수",
                        "summary":"리뷰 요약"
                    }
                    ]
                }
                }

    """

    return summ_act(text, system_message)

### 요약 실행

In [112]:
summ_fre_act['actions'] = summ_fre_act.progress_apply(
    lambda x: dynamic_summary(x['sub_category'], x['review_text']),
    axis=1
)

 12%|█▏        | 30/251 [00:36<03:55,  1.07s/it]

반환 실패,,,재시도 중
반환 실패,,,재시도 중


 12%|█▏        | 31/251 [00:39<05:55,  1.62s/it]

반환 실패,,,재시도 중


 15%|█▍        | 37/251 [00:47<04:50,  1.36s/it]

...

 69%|██████▊   | 172/251 [04:36<01:46,  1.35s/it]

반환 실패,,,재시도 중
반환 실패,,,재시도 중


 73%|███████▎  | 182/251 [04:59<02:08,  1.86s/it]

반환 실패,,,재시도 중


100%|██████████| 251/251 [06:48<00:00,  1.63s/it]


In [113]:
result = []

for idx,row in summ_fre_act.iterrows():
    style_code = row.get('style_code')
    six_p = row.get('six_p')
    sub_category = row.get('sub_category')

    if bool(row['actions']):
        actions = row.get('actions')
        for act in actions:
            try:
                freq = act.get('freq')
                summary = act.get('summary')
                result.append({
                    'style_code' : style_code,
                    'six_p' : six_p,
                    'sub_category' : sub_category,
                    'freq' : freq,
                    'summary' : summary
                })

            except:
                pass
    else:
        result.append({
            'style_code' : style_code,
            'six_p' : six_p,
            'sub_category' : sub_category,
            'freq' : '없음',
            'summary' : '없음'
        })
        
actions = pd.DataFrame(result)

In [114]:
final_df

,style_code,product_name,channel,review_text,six_p,six_tmp,sub_category,positive
0,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,TPO,False,없음,없음
1,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,광고,False,없음,없음
2,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,디자인,False,없음,없음
3,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,배송/재고,False,없음,없음
4,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,품질,False,없음,없음
...,...,...,...,...,...,...,...,...
12070,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,광고,False,없음,없음
12071,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,디자인,True,디자인,긍정
12072,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,디자인,True,핏/착용감,긍정
12073,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,배송/재고,False,없음,없음


In [115]:
rad_df = pd.merge(final_df,actions,how = 'left', on = ['style_code','six_p','sub_category'])
rad_df[['freq', 'summary']] = rad_df[['freq', 'summary']].fillna('없음')
rad_df.reset_index(drop=True)

,style_code,product_name,channel,review_text,six_p,six_tmp,sub_category,positive,freq,summary
0,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,TPO,False,없음,없음,없음,없음
1,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,광고,False,없음,없음,없음,없음
2,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,디자인,False,없음,없음,없음,없음
3,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,배송/재고,False,없음,없음,없음,없음
4,CVCKE4901N,[8만장돌파] 울혼방 베이직 촉감니트 가디건_CVCKE4901N,elandmall,1+1 너무 좋아요,품질,False,없음,없음,없음,없음
...,...,...,...,...,...,...,...,...,...,...
12070,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,광고,False,없음,없음,없음,없음
12071,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,디자인,True,디자인,긍정,8,"귀여운 디자인과 부드러운 털로 유니크하며, 따뜻하고 가벼워서 착용감이 좋습니다."
12072,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,디자인,True,핏/착용감,긍정,5,S사이즈가 잘 맞고 착용감이 좋으며 따뜻하고 귀여운 디자인입니다.
12073,SPJPE4VG01,퍼 탈착 푸퍼_SPJPE4VG01,mstore,휘뚤마뚤 잘 입고다녀요. 가벼운데 따뜻하기 까지~ 귀여운 옷입니다,배송/재고,False,없음,없음,없음,없음


In [116]:
rad_df.to_excel('D:/업무 파일/한중팀/1.CRD/RAD 버전업/2. 상세_시즌/정답지/반복시행3+.xlsx')